<a href="https://colab.research.google.com/github/DanielHevdeli/hafifot-tiug/blob/main/BERT_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/DanielHevdeli/hafifot-tiug.git

Cloning into 'hafifot-tiug'...
remote: Enumerating objects: 76, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 76 (delta 29), reused 33 (delta 5), pack-reused 0 (from 0)
Receiving objects: 100% (76/76), 7.55 MiB | 7.55 MiB/s, done.
Resolving deltas: 100% (29/29), done.


In [1]:
import pandas as pd

In [3]:
posts_df = pd.read_csv("./hafifot-tiug/data/split_data/present.csv")

In [9]:
print(f"Num of posts: {len(posts_df)}")

Num of posts: 4000


In [8]:
posts_df.head(2)

,question_id,length,date,text
0,114678,903,2016-04-10 20:05:00,"שלום , אני מאוד מקווה שתוכלו לעזור לי אני לא י..."
1,116349,888,2016-04-24 19:56:00,היי כולם \nיש לי בעיה הקשורה לתספורת שלי. \nאמ...


In [10]:
labels_df = pd.read_csv("./hafifot-tiug/data/labels/present/dummy-gpt-4_labels.csv")

In [11]:
print(f"Num of labels: {len(labels_df)}")

Num of labels: 4000


In [12]:
labels_df.head(2)

,question_id,label
0,114678,suicidal
1,116349,non-suicidal


In [13]:
merged_df = pd.merge(posts_df[['question_id', 'text']], labels_df, on='question_id')
posts_with_labels_df = merged_df[['question_id', 'text', 'label']]
display(posts_with_labels_df.head())

,question_id,text,label
0,114678,"שלום , אני מאוד מקווה שתוכלו לעזור לי אני לא י...",suicidal
1,116349,היי כולם \nיש לי בעיה הקשורה לתספורת שלי. \nאמ...,non-suicidal
2,108590,הגעתי לאתר הזה במקרה אחרי שקראתי טיפה מפוסטים ...,non-suicidal
3,100709,"בערך מגיל 17 אני סובלת מחרדה, כאשר חלה החרפה ק...",suicidal
4,120766,"אני תוהה כבר כמה שנים מה ללמוד בעתיד, איזה מקצ...",non-suicidal


In [14]:
print(f"posts_with_labels_df len: {len(posts_with_labels_df)}")

posts_with_labels_df len: 4000


# Create a BERT Classifier